In [1]:
#importar librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model


In [3]:
# CarGa del dataSET
data_path = '.\genome.csv ' #ruta del modelo csv 
data = pd.read_csv(data_path, sep='\t')

# ostrar las primeras filas del dataset
print(data.head())

# Información sobre el dataset
print(data.info())
print(data.describe())


  rsid,"chromosome","position","genotype"
0                rs4477212,"1",72017,"AA"
1               rs3094315,"1",742429,"AA"
2               rs3131972,"1",742584,"GG"
3              rs12124819,"1",766409,"AA"
4              rs11240777,"1",788822,"AG"
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966977 entries, 0 to 966976
Data columns (total 1 columns):
 #   Column                                   Non-Null Count   Dtype 
---  ------                                   --------------   ----- 
 0   rsid,"chromosome","position","genotype"  966977 non-null  object
dtypes: object(1)
memory usage: 7.4+ MB
None
       rsid,"chromosome","position","genotype"
count                                   966977
unique                                  966977
top                   rs4477212,"1",72017,"AA"
freq                                         1
